<center><img src=img/MScAI_brand.png width=70%></center>

# Object-Oriented Programming in Python

This notebook is about OOP in Python. Students who have not studied OOP concepts such as objects, class, constructors, methods, inheritance, and polymorphism in another language should refer to Downey, Chapters 15--18, and feel free to contact me for further resources and help. 

### Contents

* Defining classes
* Special double-underscore methods
* Inheritance and duck-typing

In Python, we define a class using `class`. The following, believe it or not, is a working class which can be instantiated.

In [1]:
class C:
    pass

Now we can make an object `c` of type `C`. The **constructor** is a special method (a **method** is a function that belongs to a class or object, rather than a stand-alone function). Its job is to **create an instance**. We call the constructor as if we were calling the class itself:

In [2]:
c = C()

In [3]:
print(c) # it exists, but it doesn't do much

Now let's make a better constructor. The `self` is the first argument of all methods and it refers to the object itself -- the equivalent of `this` in some languages.

In [4]:
class C:
    def __init__(self, data=17):
        self.data = data
c = C()
print(c.data)

17


In Python, we don't have any restrictions on accessing fields, and notice, objects are mutable.

In [5]:
c.data += 1 # no need for set() or get()
print(c.data)

18


A class has its own namespace. So it's fine to have a variable called `data` and a variable called `c.data` at the same time with different values.

### Double-underscore methods

The `__init__` constructor is just one of several special double-underscore methods in Python. When we use some common Python things like `len` and `<`, it results in a call to a particular method, for example:

call      |  translation
----------|------------------------
`C(data)` |  `C.__init__(data)`
`len(c)`  |  `c.__len__()`
`repr(c)` |  `c.__repr__()`
`str(c)`  |  `c.__str__()` 
`c < d`   |  `c.__lt__(d)` 

It's worth remarking on the relationship between `str(c)` and `repr(c)`. 

When we call `print(c)`, it calls `str(c)` and that is translated to `c.__str__()` which should return a printable, human-readable string.

When we just put `c` by itself in IPython or a Jupyter Notebook cell, `repr(c)` is called and that is translated to `c.__repr__()` which should return a printable string representing everything there is to know about the object, eg all the values of variables which would be needed to re-created the object. 

Often developers try to implement `__repr__` so that the result would work as a constructor call, eg:

In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=7)
dt # equivalent to calling `repr(dt)`

DecisionTreeClassifier(max_depth=7)

The result of `repr(dt)` could be copied and pasted as valid Python code.

A slight complication: 

* If an object `c` doesn't have `__repr__`, then `repr(c)` will give an ugly default like `<__main__.C at 0x27597d9fc18>`.
* If an object `c` doesn't have `__str__`, then `str(c)` will fallback to `c.__repr__()`. 

For most classes, the `repr` string ("everything there is to know about the object") also works fine as a *human-readable* string. So we need only implement `__repr__`. 

If we really need two different strings for these purposes, we can implement both `__repr__` and `__str__`.

So, let's implement some of these "dunder" (double-underscore) methods:

In [7]:
class C:
    def __init__(self, data=17):
        self.data = data
    def __repr__(self):
        return f"C({self.data})"
    def __lt__(self, other):
        return self.data < other.data
        
c1 = C()
c2 = C(18)
print(c1 < c2)

True


### A free lunch

Sometimes implementing one special method is enough to get others "for free". We implemented `__lt__` for "less than", allowing `c1 < c2` to work. We didn't implement `__gt__`. Nevertheless, it works:

In [8]:
c1 > c2

False

That's because Python knows that these operators are symmetric. If we wanted to define `>` to be non-symmetric to `<`, we could do so, just by implementing both `__lt__` and `__gt__` separately.

### Inheritance

We can make a new class which has some behaviour (methods and fields) of its own, and inherits other behaviour from a superclass. The syntax is: put the parent (superclass) in parentheses after `class C`.

In [9]:
class D(C):
    def hello(self):
        print(f"Hello, my value is {self.data}")

This new class has composite behaviour:

In [10]:
d = D(12)
print(d) # inherited from C
d.hello() # in D itself

C(12)
Hello, my value is 12


### `super`

Sometimes we want to inherit a method, but also add to it. This is especially common in `__init__`. We call `super` to get the **superclass** (the class we are inheriting from), and call its `__init__` directly. 

In [11]:
class D(C):
    def __init__(self, data=17):
        super().__init__(data)
        self.config = "specific to D"
    def hello(self):
        print(f"Hello, my value is {self.data} and my config is {self.config}")
d = D(15)
d.hello()

Hello, my value is 15 and my config is specific to D


### Duck typing

Don't forget, we have duck typing! In fact, duck typing is the Python approach to *polymorphism*. For example this works, even though `d` and `c` are of different types, `D` and `C`:

In [12]:
d < c

True

We can understand this by the substitution model:

```
d < c ... gets translated to ...
d.__lt__(c) ... which runs ...
d.data < c.data ... which evaluates to ...
12 < 17 ... which evaluates to ...
True
```